# Create a Chatbot using NLP and TensorFlow
use case is for a coffee shop

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import json
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

import nltk
nltk.download('wordnet')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import files
upload = files.upload()

Saving intents.json to intents (2).json


In [ ]:
intents = pd.read_json("intents.json")

In [ ]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# Loop through Intents file to convert
# into words, classes, documents and ignore letters
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# Stemming, Lowering and Removing Duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [ ]:
# Prepare Training Data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [stemmer.stem(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

x_train= list(training[:, 0])
y_train = list(training[:, 1])

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               6272      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 9)                 585       
                                                                 
Total params: 15,113
Trainable params: 15,113
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')

Epoch 1/200
6/6 [==============================] - 1s 4ms/step - loss: 2.2714 - accuracy: 0.0741
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 2.2580 - accuracy: 0.0741
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 2.2453 - accuracy: 0.1111
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 2.2023 - accuracy: 0.1852
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 2.1552 - accuracy: 0.2222
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 2.1710 - accuracy: 0.1852
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 2.1392 - accuracy: 0.1111
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0578 - accuracy: 0.2593
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 2.0268 - accuracy: 0.3704
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0818 - accuracy: 0.1481
Epoch 11/200
6/6 [===========

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result


In [ ]:
def chatbot_response(text):
    intents_list = predict_class(text, model)
    response = get_response(intents_list, intents)
    return response

# Testing the chatbot
print(chatbot_response("Hi"))


1/1 [==============================] - 0s 176ms/step
Good to see you again
